In [1]:
import json
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import ollama
from sympy import simplify
from sympy.parsing.latex import parse_latex
from tqdm import tqdm

# Глобальная переменная для отслеживания вывода моделей
_models_listed = False


def extract_answer_with_ollama(generated_answer: str, model_name: str = "llama3.2:latest") -> str:
    global _models_listed

    # Показать список моделей при первом вызове
    if not _models_listed:
        try:
            models = ollama.list()
            print("📦 Загруженные модели в Ollama:")
            for m in models['models']:
                # Используем 'model' вместо 'name'
                model = m['model']
                size = m.get('size', 'неизвестно')
                modified = m.get('modified_at', 'неизвестно')
                print(f"  - {model} (size: {size}, modified: {modified})")
            _models_listed = True
        except Exception as e:
            print(f"❌ Не удалось получить список моделей: {e}")
            _models_listed = True  # чтобы не повторять попытку

    # Проверка на пустой ответ
    if not generated_answer.strip():
        return ""

    prompt = f"""
You are a mathematics assistant specialized in differential equations.
Your task: extract the final solution expression from the provided reasoning.
Follow these rules:

- Prefer expressions inside \\boxed{{...}}, but extract the raw expression without the \\boxed{{}} itself.
- If no \\boxed{{}} found, extract the last complete solution (such as y(x) = ...).
- Do NOT generate or modify any math. Return exactly as found.
- Preserve LaTeX formatting, but remove \\boxed{{}} if present.
- Return only the solution expression, without any explanations.

Text:
\"\"\"
{generated_answer}
\"\"\"

Final Solution:
"""

    try:
        response = ollama.generate(model=model_name, prompt=prompt)
        extracted_answer = response.get("response", "").strip()
        return clean_extracted_latex(extracted_answer)
    except Exception as e:
        print(f"❌ Ошибка Ollama: {e}")
        return ""



# ———————————————— Постобработка и нормализация ———————————————— #
def clean_extracted_latex(extracted_answer: str) -> str:
    if not extracted_answer:
        return ""

    result = extracted_answer.strip()

    # Удаление LaTeX-окружений
    result = re.sub(r'\\\[|\\\]', '', result)
    result = re.sub(r'\$\$|\$', '', result)
    result = re.sub(r'\\boxed\{([^{}]*)\}', r'\1', result)

    # Удаление начального y(x)= или y= (с пробелами)
    result = re.sub(r'^(y\s*(\([a-z]\))?\s*=)', '', result, flags=re.IGNORECASE)

    # Удаление всех пробелов
    result = re.sub(r'\s+', '', result)

    return result


def normalize_latex_expression(latex_str: str, target_var: str = 'x') -> str:
    if not latex_str:
        return ""

    # Унификация констант: C_{1}, C_1 → C
    latex_str = re.sub(r'C(_\{?\d+\}?)?', 'C', latex_str)

    # Унификация переменных внутри экспонент e^{...}
    def replace_exp_vars(m):
        content = m.group(1)
        replaced = re.sub(r'(?<![_\\])[a-zA-Z]', target_var, content)
        return f"e^{{{replaced}}}"

    latex_str = re.sub(r'e\^\{([^}]*)\}', replace_exp_vars, latex_str)

    # Унификация всех одиночных строчных переменных вне экспонент, исключая 'e'
    def replace_vars(m):
        ch = m.group(0)
        if ch in ['C', 'K', 'A', 'e']:
            return ch
        return target_var

    latex_str = re.sub(r'(?<![_\\])[a-z]', replace_vars, latex_str)

    return latex_str


def unify_constants_before_exponent(latex_str: str) -> str:
    if not latex_str:
        return ""
    return re.sub(r'C(_\{?\d+\}?)', 'C', latex_str)


def tokenize_latex(text: str) -> list[str]:
    return list(text.replace(' ', '').strip())


# ———————————————— Символьное сравнение через SymPy ———————————————— #
def are_expressions_symbolically_equivalent(expr1: str, expr2: str) -> bool:
    try:
        sym_expr1 = simplify(parse_latex(expr1))
        sym_expr2 = simplify(parse_latex(expr2))
        return sym_expr1.equals(sym_expr2)
    except Exception as e:
        print(f"⚠️ Ошибка в символьном сравнении: {e}")
        return False


# ———————————————— Основная функция оценки BLEU и эквивалентности ———————————————— #
def evaluate_bleu_with_ollama(file_path: str, model_name: str = "qwen2.5:latest",
                              limit=None, smoothing_method=2, verbose=True):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if limit is not None:
        data = data[:limit]

    smoothing_function = SmoothingFunction().method2
    output_results = []

    for idx, item in enumerate(tqdm(data, desc="Обработка примеров", unit="пример")):
    #for idx, item in enumerate(data):
        true_answer = item.get("true_answer", "").strip()
        generated_answer = item.get("generated_answer", "").strip()
        problem = item.get("equation") or item.get("problem") or item.get("question") or f"example_{idx}"
        type_eq = item.get("type_eq", "").strip()

        if not true_answer or not generated_answer:
            continue

        extracted_answer = extract_answer_with_ollama(generated_answer, model_name)

        if not extracted_answer:
            output_results.append({
                "original_equation": problem,
                "type_equation": type_eq,
                "true_answer": true_answer,
                "extracted_answer": "",
                "bleu_score": 0.0,
                "exact_match": False
            })
            continue

        # Определяем переменную для унификации (по экспоненте в истинном ответе)
        target_var_match = re.search(r'e\^\{[^}]*?([a-zA-Z])[^}]*?\}', true_answer)
        target_var = target_var_match.group(1) if target_var_match else 'x'

        true_normalized = normalize_latex_expression(clean_extracted_latex(true_answer), target_var=target_var)
        extracted_normalized = normalize_latex_expression(clean_extracted_latex(extracted_answer), target_var=target_var)

        # Активируем унификацию констант перед экспонентой
        true_normalized = unify_constants_before_exponent(true_normalized)
        extracted_normalized = unify_constants_before_exponent(extracted_normalized)

        reference = [tokenize_latex(true_normalized)]
        hypothesis = tokenize_latex(extracted_normalized)
        bleu_score = sentence_bleu(reference, hypothesis, smoothing_function=smoothing_function)

        is_exact_match = (true_normalized == extracted_normalized)

        output_results.append({
            "original_equation": problem,
            "type_equation": type_eq,
            "true_answer": true_normalized,
            "extracted_answer": extracted_normalized,
            "bleu_score": round(bleu_score, 4),
            "exact_match": is_exact_match
        })

        #if verbose:
        #    print(f"\n🔢 Пример {idx + 1}")
        #    print(f"📄 Уравнение: {problem}")
        #    print(f"📄 Тип уравнения: {type_eq}")
        #    print(f"✅ Истинный ответ (нормализованный): {true_normalized}")
        #    print(f"🧠 Извлечённый ответ (нормализованный): {extracted_normalized}")
        #    print(f"📊 BLEU: {bleu_score:.4f}")
    
        if verbose and idx % 100 == 0:  # Реже выводим детали
            tqdm.write(f"\n🔢 Пример {idx + 1}")
            tqdm.write(f"📄 Тип уравнения: {type_eq}")
            tqdm.write(f"✅ Истинный ответ (нормализованный): {true_normalized}")
            tqdm.write(f"🧠 Извлечённый ответ (нормализованный): {extracted_normalized}")
            tqdm.write(f"📊 BLEU: {bleu_score:.4f}")

    total = len(output_results)
    avg_bleu = sum([r["bleu_score"] for r in output_results]) / total if total > 0 else 0

    print("\n---")
    print(f"🧠 Средний BLEU: {avg_bleu:.4f} на {total} примерах")

    output_file = file_path.replace(".json", "_results_ollama.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_results, f, indent=2, ensure_ascii=False)
    print(f"\n💾 Результаты сохранены в {output_file}")

    # Расчёт статистики по типам уравнений
    calculate_stats_by_type(output_file)


# ———————————————— Функция расчёта статистики по типам уравнений ———————————————— #
def calculate_stats_by_type(results_file: str):
    with open(results_file, "r", encoding="utf-8") as f:
        results = json.load(f)

    stats = {}

    for item in results:
        eq_type = item["type_equation"]
        bleu_score = item["bleu_score"]
        exact_match = item["exact_match"]

        if eq_type not in stats:
            stats[eq_type] = {
                "count": 0,
                "total_bleu": 0.0,
                "exact_matches": 0
            }

        stats[eq_type]["count"] += 1
        stats[eq_type]["total_bleu"] += bleu_score
        if exact_match:
            stats[eq_type]["exact_matches"] += 1

    final_stats = {}
    for eq_type, data in stats.items():
        avg_bleu = data["total_bleu"] / data["count"]
        exact_ratio = data["exact_matches"] / data["count"]
        final_stats[eq_type] = {
            "count": data["count"],
            "avg_bleu": round(avg_bleu, 4),
            "exact_matches": data["exact_matches"],
            "exact_ratio": round(exact_ratio, 4)
        }

    print("\n📊 Статистика по типам уравнений:")
    for eq_type, data in final_stats.items():
        print(f"\n🧩 Тип: {eq_type}")
        print(f"  Количество примеров: {data['count']}")
        print(f"  Средний BLEU: {data['avg_bleu']:.4f}")
        print(f"  Точные совпадения: {data['exact_matches']} ({data['exact_ratio']:.2%})")

    # Сохранение статистики в файл
    stats_file = results_file.replace("_results_ollama.json", "_stats_by_type.json")
    with open(stats_file, 'w', encoding='utf-8') as f:
        json.dump(final_stats, f, indent=2, ensure_ascii=False)
    print(f"\n💾 Статистика по типам сохранена в {stats_file}")


# ———————————————— Запуск ———————————————— #
if __name__ == "__main__":
    file_path = r'./Qwen_1_5b_inference_results_2000tok_balansed.json'  # Путь к вашему файлу
    model_name = "qwen2.5:latest"  # Модель Ollama

    evaluate_bleu_with_ollama(file_path, model_name=model_name, limit=1709)

Обработка примеров:   0%|                                                                 | 0/1709 [00:00<?, ?пример/s]

📦 Загруженные модели в Ollama:
  - nomic-embed-text:latest (size: 274302450, modified: 2025-06-05 19:05:05.791478+03:00)
  - deepseek-r1:14b (size: 8988112209, modified: 2025-06-02 14:40:21.868115+03:00)
  - qwen2.5:latest (size: 4683087332, modified: 2025-05-29 18:17:21.963823+03:00)
  - llama3.2:1b (size: 1321098329, modified: 2025-04-15 16:46:59.121711+03:00)
  - gemma3:1b (size: 815319791, modified: 2025-04-15 16:36:59.620343+03:00)
  - cogito:3b (size: 2241013645, modified: 2025-04-09 20:51:29.521689+03:00)
  - deepseek-r1:1.5b (size: 1117322599, modified: 2025-04-04 15:20:36.186802+03:00)
  - llama3.2:latest (size: 2019393189, modified: 2025-04-04 14:26:14.639084+03:00)


Обработка примеров:   0%|                                                       | 1/1709 [00:06<3:11:14,  6.72s/пример]


🔢 Пример 1
📄 Тип уравнения: homogenous 3nd order
✅ Истинный ответ (нормализованный): Ce^{-8x}+Ce^{-10x}+Ce^{-3x}
🧠 Извлечённый ответ (нормализованный): Ce^{-4x}+Ce^{-5x}+Ce^{-12x}
📊 BLEU: 0.7093


Обработка примеров:   6%|███▏                                                 | 101/1709 [05:57<1:03:36,  2.37s/пример]

...


🔢 Пример 1601
📄 Тип уравнения: homogenous 2nd order
✅ Истинный ответ (нормализованный): Ce^{-24x}+Ce^{-17x}
🧠 Извлечённый ответ (нормализованный): Ce^{-17x}+Ce^{-24x}
📊 BLEU: 0.9850


Обработка примеров: 100%|███████████████████████████████████████████████████▊| 1701/1709 [6:46:06<00:52,  6.62s/пример]


🔢 Пример 1701
📄 Тип уравнения: homogenous 2nd order
✅ Истинный ответ (нормализованный): Ce^{28x}+Ce^{-38x}
🧠 Извлечённый ответ (нормализованный): Ce^{28x}+Ce^{-38x}
📊 BLEU: 1.0000


Обработка примеров: 100%|████████████████████████████████████████████████████| 1709/1709 [6:46:52<00:00, 14.28s/пример]


---
🧠 Средний BLEU: 0.6034 на 1709 примерах

💾 Результаты сохранены в ./Qwen_1_5b_inference_results_2000tok_balansed_results_ollama.json

📊 Статистика по типам уравнений:

🧩 Тип: homogenous 3nd order
  Количество примеров: 341
  Средний BLEU: 0.8225
  Точные совпадения: 32 (9.38%)

🧩 Тип: separable variables
  Количество примеров: 342
  Средний BLEU: 0.2527
  Точные совпадения: 0 (0.00%)

🧩 Тип: polynomial
  Количество примеров: 342
  Средний BLEU: 0.5466
  Точные совпадения: 39 (11.40%)

🧩 Тип: homogenous 2nd order
  Количество примеров: 342
  Средний BLEU: 0.9701
  Точные совпадения: 148 (43.27%)

🧩 Тип: inhomogenous 2nd order
  Количество примеров: 342
  Средний BLEU: 0.4258
  Точные совпадения: 0 (0.00%)

💾 Статистика по типам сохранена в ./Qwen_1_5b_inference_results_2000tok_balansed_stats_by_type.json
